# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [2]:
# !pip install scikit-surprise

간단한 surprise 실습

In [ ]:
import numpy as np

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [ ]:

model = SVD()
cross_validate(model, data, measures = ['rmse','mae'], cv =5, verbose = True)

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천

* 유사도를 기반으로 추천
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능   

* 유사도 계산 방법 : 3가지    
  *인진벡터의 내적 : 중복갯수   
  *유클리드 거리 :    
  *코사인 유사도 :    

#### 데이터 로드

In [40]:
data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data =  np.array(data.raw_ratings, dtype = int)

In [41]:
raw_data 
# user_id, movie_id, reating, time

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])



#### 이진벡터의 내적 을 통한 추천
- 내가 본 영화들과 중복이 많은 유저 선택 (나와 취향이 비슷한 유저 선택)
- 나와 유사도가 높은 유저의 영화들중, 내가 보지 않은 영화 목록 추천

1. 인접행렬 생성 (user_id*movie_id)
2. 유사도 구하기 
3. 유사도가 높은 id 및 유사도 반환
4. 유사도 높은 유저목록에서 내가 보지 않은 영화목록 추천

In [42]:
# id가 배열인덱스와 일치하도록 -> 0부터 시작할 수 있게 조정 --> 각 id를 행x열로 표시하기 위해
raw_data[:,0] -= 1 # user_id
raw_data[:,1] -= 1 # movie_id

In [43]:
# 인접행렬의 크기 계산 
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:,1] )
shape = (n_users+1, n_movies+1)

In [44]:
shape
# user_id 최대값 , movie_id 최대값  --> 이것으로 matrix 생성

(943, 1682)

In [45]:
# 인접행렬 생성
# user_id - movie_id 와의 관계를 matrix로 표현
# 1 : 평가 데이터 있음 
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] =1 

adj_matrix


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [47]:
# 테스트값 생성
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1 ,[]

#user_vector : 사용자가 평가한 movie_id 벡터 
for user_id, user_vector in enumerate(adj_matrix) : 
    if my_id != user_id: # 나 자신이 아닌 다른 사용자와 값 비교
        similarity = np.dot(my_vector, user_vector) # 유사도 계산산
        if similarity > best_match : 
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 183, Best Match ID : 275


In [48]:
# 추천 리스트 뽑기 
recommend_list = []
for i , log in enumerate(zip(my_vector, best_match_vector)) : 
    log1, log2 = log
    if log1 < 1. and log2 > 0. : #log1 < 1 -> 내가 보지 않은 영화 , log2 > 0 : 다른 유저가 본 영화
        recommend_list.append(i)

print('best_match movies : ', recommend_list)

best_match movies :  [272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736

#### 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [49]:
# 테스트값 생성
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1 ,[]

#user_vector : 사용자가 평가한 movie_id 벡터 
for user_id, user_vector in enumerate(adj_matrix) : 
    
    if my_id != user_id: # 나 자신이 아닌 다른 사용자와 값 비교
        
        similarity = np.sqrt(sum(np.square(my_vector-user_vector))) # 유클리디안 유사도 적용용
        
        if similarity < best_match : 
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 14.832396974191326, Best Match ID : 737


In [50]:
# 추천 리스트 뽑기 
recommend_list = []
for i , log in enumerate(zip(my_vector, best_match_vector)) : 
    log1, log2 = log
    if log1 < 1. and log2 > 0. : #log1 < 1 -> 내가 보지 않은 영화 , log2 > 0 : 다른 유저가 본 영화
        recommend_list.append(i)

print('best_match movies : ', recommend_list)

best_match movies :  [297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [51]:
def compute_cos_similarity(v1,v2) : 
    norm1 = np.sqrt(np.sum(np.square(v1))) #벡터크기 계산
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1,v2)
    return dot/(norm1*norm2)

In [52]:
# 테스트값 생성
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1 ,[]

#user_vector : 사용자가 평가한 movie_id 벡터 
for user_id, user_vector in enumerate(adj_matrix) : 
    
    if my_id != user_id: # 나 자신이 아닌 다른 사용자와 값 비교
        
        similarity = compute_cos_similarity(my_vector,user_vector) # 코사인유사도 적용
        if similarity > best_match : 
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.5278586163659506, Best Match ID : 915


In [53]:
# 추천 리스트 뽑기 
recommend_list = []
for i , log in enumerate(zip(my_vector, best_match_vector)) : 
    log1, log2 = log
    if log1 < 1. and log2 > 0. : #log1 < 1 -> 내가 보지 않은 영화 , log2 > 0 : 다른 유저가 본 영화
        recommend_list.append(i)

print('best_match movies : ', recommend_list)

best_match movies :  [272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


### 기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [54]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data : 
   adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [55]:
# 테스트값 생성
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1 ,[]

#user_vector : 사용자가 평가한 movie_id 벡터 
for user_id, user_vector in enumerate(adj_matrix) : 
    
    if my_id != user_id: # 나 자신이 아닌 다른 사용자와 값 비교
        
        similarity = np.sqrt(sum(np.square(my_vector-user_vector))) # 유클리디안 유사도 적용용
        
        if similarity < best_match : 
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 55.06359959174482, Best Match ID : 737


In [56]:
# 테스트값 생성
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1 ,[]

#user_vector : 사용자가 평가한 movie_id 벡터 
for user_id, user_vector in enumerate(adj_matrix) : 
    
    if my_id != user_id: # 나 자신이 아닌 다른 사용자와 값 비교
        
        similarity = compute_cos_similarity(my_vector,user_vector) # 코사인유사도 적용
        if similarity > best_match : 
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector
            
print('Best Match : {}, Best Match ID : {}'.format(best_match, best_match_id))

Best Match : 0.569065731527988, Best Match ID : 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움   

* KNNBasic, SVD, SVDpp, NMF 

In [59]:
from surprise import KNNBasic, SVD,SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [60]:
data  = Dataset.load_builtin('ml-100k',prompt=False)

#### KNN을 사용한 협업 필터링

In [61]:
model = KNNBasic()
cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, n_jobs = 4, verbose=True )

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9774  0.9785  0.9802  0.9817  0.9767  0.9789  0.0019  
MAE (testset)     0.7710  0.7741  0.7740  0.7746  0.7703  0.7728  0.0018  
Fit time          0.64    0.59    0.56    0.53    0.41    0.54    0.08    
Test time         2.64    2.61    2.58    2.59    2.03    2.49    0.23    


{'test_rmse': array([0.97736471, 0.97845821, 0.98023125, 0.98172738, 0.9766726 ]),
 'test_mae': array([0.77098487, 0.77406631, 0.77395541, 0.77463597, 0.77033718]),
 'fit_time': (0.6383116245269775,
  0.5927672386169434,
  0.5572094917297363,
  0.5294249057769775,
  0.4062654972076416),
 'test_time': (2.635643482208252,
  2.606750011444092,
  2.5819430351257324,
  2.5942628383636475,
  2.029741048812866)}

#### SVD를 사용한 협업 필터링

In [62]:
model = SVD()
cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, n_jobs = 4, verbose=True )

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9414  0.9362  0.9369  0.9341  0.9325  0.9362  0.0030  
MAE (testset)     0.7436  0.7389  0.7396  0.7339  0.7318  0.7376  0.0042  
Fit time          0.70    0.77    0.78    0.77    0.66    0.74    0.05    
Test time         0.18    0.18    0.19    0.16    0.15    0.17    0.02    


{'test_rmse': array([0.94140932, 0.93621831, 0.93691399, 0.93408529, 0.93253441]),
 'test_mae': array([0.74358032, 0.7388997 , 0.73958978, 0.73393853, 0.73184174]),
 'fit_time': (0.7040596008300781,
  0.7730977535247803,
  0.7751011848449707,
  0.7727682590484619,
  0.655402660369873),
 'test_time': (0.17792749404907227,
  0.18022847175598145,
  0.19193768501281738,
  0.16078472137451172,
  0.1476449966430664)}

#### NMF를 사용한 협업 필터링

In [63]:
model = NMF()
cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, n_jobs = 4, verbose=True )

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9648  0.9583  0.9621  0.9613  0.9737  0.9640  0.0052  
MAE (testset)     0.7601  0.7514  0.7573  0.7561  0.7661  0.7582  0.0048  
Fit time          1.47    1.50    1.53    1.55    1.17    1.44    0.14    
Test time         0.15    0.16    0.16    0.15    0.12    0.15    0.02    


{'test_rmse': array([0.9647505 , 0.95831231, 0.96208166, 0.96130728, 0.97365705]),
 'test_mae': array([0.76013547, 0.75142709, 0.75733132, 0.75609144, 0.76605495]),
 'fit_time': (1.4730098247528076,
  1.4967966079711914,
  1.5252594947814941,
  1.554523229598999,
  1.167973518371582),
 'test_time': (0.15141534805297852,
  0.16353082656860352,
  0.16140961647033691,
  0.1493971347808838,
  0.11777520179748535)}

* SVD++를 사용한 협업 필터링

In [64]:
model = SVDpp()
cross_validate(model, data, measures=['rmse', 'mae'], cv = 5, n_jobs = 4, verbose=True )

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9262  0.9173  0.9085  0.9203  0.9280  0.9200  0.0070  
MAE (testset)     0.7277  0.7177  0.7116  0.7220  0.7279  0.7214  0.0062  
Fit time          21.42   21.26   21.65   21.66   16.42   20.48   2.04    
Test time         4.22    4.26    4.28    4.22    3.21    4.04    0.41    


{'test_rmse': array([0.92616116, 0.91729   , 0.90846652, 0.92030259, 0.92801886]),
 'test_mae': array([0.72769514, 0.71772011, 0.7115888 , 0.72195597, 0.72785053]),
 'fit_time': (21.416019916534424,
  21.25504231452942,
  21.646311283111572,
  21.658692598342896,
  16.418633699417114),
 'test_time': (4.217417001724243,
  4.255372762680054,
  4.275878190994263,
  4.215004920959473,
  3.2116012573242188)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [71]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [74]:
data  = Dataset.load_builtin('ml-100k',prompt=False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:,0] -=1
raw_data[:,1] -=1

In [75]:
n_users = np.max(raw_data[:,0])
n_movies= np.max(raw_data[:,1])
shape = (n_users+1, n_movies+1)
print(shape)

(943, 1682)


In [76]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating , time in raw_data : 
    adj_matrix[user_id][movie_id] = rating

In [77]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [78]:
U, S, V = randomized_svd(adj_matrix, n_components=2)
S = np.diag(S)

In [79]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


(943, 2)
(2, 2)
(2, 1682)


array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

Best Match: 0.9999942295956324, Best Match ID: 235


[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

Best Match: 0.9999999951364141, Best Match ID: 1287


[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_nmf.py:1035: FutureWarning: The default value of init will change from random to None in 0.23 to make it consistent with decomposition.NMF.
  "with decomposition.NMF.", FutureWarning)


array([[3.71104386e+00, 1.48478414e+00, 7.39554801e-01, ...,
        3.64526020e-03, 1.45529217e-02, 1.44127541e-02],
       [2.11740926e+00, 2.36733594e-01, 5.51609827e-01, ...,
        4.76715033e-03, 2.44095997e-05, 0.00000000e+00],
       [9.85437620e-01, 1.10175294e-01, 2.56718002e-01, ...,
        2.21862129e-03, 1.13601741e-05, 0.00000000e+00],
       ...,
       [1.04462665e+00, 1.16792830e-01, 2.72137435e-01, ...,
        2.35187989e-03, 1.20425082e-05, 0.00000000e+00],
       [1.45791701e+00, 5.42098835e-01, 2.99293177e-01, ...,
        1.61349670e-03, 5.15831117e-03, 5.10699643e-03],
       [2.44660591e+00, 9.41428352e-01, 4.95526147e-01, ...,
        2.56812371e-03, 9.08641597e-03, 8.99741614e-03]])

* 사용자 기반 추천

In [ ]:
my_id, my_vector = 0, U[0]  # 내 id가 0번 이라고 가정
b

Best Match: 0.99999996802728, Best Match ID: 166


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[287, 289, 317, 363, 380, 391, 403, 434, 464, 477, 485, 492, 511, 512, 520, 529, 553, 567, 602, 605, 614, 640, 654, 658, 672, 673, 674, 697, 718, 725, 732, 734, 830, 948, 1124, 1125, 1146, 1199, 1224, 1303, 1304, 1305, 1306, 1307, 1308, 1309]


* 항목 기반 추천

In [ ]:
my_id, my_vector = 0, B.T[0]  # 기준 항목이 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1, []

for item_id, item_vector in enumerate(V.T):
  if my_id != item_id:
    cos_similarity = compute_cos_similarity(my_vector, item_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = item_id
      best_match_vector = item_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999346987367, Best Match ID: 497


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 